# Libraries

Import necessary libraries

In [2]:
# standard libraries
import os
import time
import random
import re
from collections import Counter
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from Bio import SeqIO
import iFeatureOmegaCLI

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (
    train_test_split, cross_val_score, StratifiedKFold, 
    GroupShuffleSplit, GridSearchCV
)
from sklearn.metrics import (
    make_scorer, accuracy_score, balanced_accuracy_score,
    classification_report, confusion_matrix, roc_auc_score,
    log_loss, precision_score, recall_score, f1_score
)
from sklearn.cluster import KMeans
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

import shap

# Functions

### Handling ambiguous nucleotides in FASTA Files

This section describes how to process DNA sequences from a FASTA file. It replaces any ambiguous bases with 'N' and saves the cleaned sequences to a new file.

In [3]:
def replace_with_N(base):
    """
    replace any nucleotide base that is not 'A', 'C', 'G', or 'T' with 'N'
    
    parameters:
    base (str): a single character representing a nucleotide
    
    returns:
    str: 'N' if the base is ambiguous; otherwise, it returns the original base
    """
    return base if base in {"A", "C", "G", "T"} else "N"

def clean_sequences_with_N(input_file, output_file):
    """
    process a fasta file by replacing ambiguous bases with 'N' and 
    writing the cleaned sequences to a new file. also returns statistics 
    on ambiguous bases.
    
    parameters:
    input_file (str): path to the input fasta file
    output_file (str): path to the output fasta file with cleaned sequences
    
    returns:
    tuple: total number of ambiguous bases and total number of bases in all sequences
    """
    total_ambiguous = 0  # total number of ambiguous characters across all sequences
    total_bases = 0  # total number of bases across all sequences

    with open(input_file, "r") as input_handle, open(output_file, "w") as output_handle:
        for record in SeqIO.parse(input_handle, "fasta"):
            sequence = str(record.seq).upper()
            
            # count ambiguous bases
            ambiguous_count = sum(1 for base in sequence if base not in {"A", "C", "G", "T"})
            total_ambiguous += ambiguous_count
            total_bases += len(sequence)
            
            # replace any character that is not A, C, G, or T with 'N'
            clean_seq = "".join(replace_with_N(base) for base in sequence)
            
            # write the cleaned sequence to the new FASTA file
            output_handle.write(f">{record.id}\n{clean_seq}\n")

    return total_ambiguous, total_bases  

### Hmmer

In [4]:
"""
# HMMER - hmmscan command
runs `hmmscan` to scan protein sequences against a Hidden Markov Model (HMM) database.

description:
    - scans the input protein sequences from `protein_sequences.fasta` against the HMM database `hmm/vog.hmm`.
    - outputs a tabular summary of matches to `results.tbl`.

inputs:
    - HMM database file: `hmm/vog.hmm`
    - protein sequences file: `protein_sequences.fasta`

outputs:
    - tabular output file: `results.tbl` (summary of matches between queries and HMM profiles).

default parameters:
    - `--E 10`: searches for matches with an E-value ≤ 10 (default threshold for reporting matches).
    - `--domE 10`: sets the E-value threshold for reporting individual domains to 10.
    - `--incE 0.01`: sets the E-value threshold for including matches in the final report to 0.01.
    - `--incdomE 0.01`: e-value threshold for including domains in the report is also 0.01.
    - `--cpu 1`: uses one CPU core for computation by default (can be adjusted with `--cpu <num>`).

customising parameters:
    to modify defaults, add additional flags to the command:
    ```bash
    !hmmscan --tblout results.tbl --cpu 4 --E 1e-5 hmm/vog.hmm protein_sequences.fasta
    ```

usage:
    Place the command in a Jupyter cell with the `!` prefix and run it.
"""

#!hmmscan --cpu 10 --domtblout features/vog_matches_domtblout.txt data/raw/phmms/vog/vog.hmm data/sequences/protein_sequences.fasta
#!hmmscan --cpu 10 --domtblout features/vfam_matches_domtblout.txt data/raw/phmms/vfam/vfam.hmm data/sequences/protein_sequences.fa

'\n# HMMER - hmmscan command\nruns `hmmscan` to scan protein sequences against a Hidden Markov Model (HMM) database.\n\ndescription:\n    - scans the input protein sequences from `protein_sequences.fasta` against the HMM database `hmm/vog.hmm`.\n    - outputs a tabular summary of matches to `results.tbl`.\n\ninputs:\n    - HMM database file: `hmm/vog.hmm`\n    - protein sequences file: `protein_sequences.fasta`\n\noutputs:\n    - tabular output file: `results.tbl` (summary of matches between queries and HMM profiles).\n\ndefault parameters:\n    - `--E 10`: searches for matches with an E-value ≤ 10 (default threshold for reporting matches).\n    - `--domE 10`: sets the E-value threshold for reporting individual domains to 10.\n    - `--incE 0.01`: sets the E-value threshold for including matches in the final report to 0.01.\n    - `--incdomE 0.01`: e-value threshold for including domains in the report is also 0.01.\n    - `--cpu 1`: uses one CPU core for computation by default (can be 

### Parsing .tblout files: extracting and filtering HMMER Results (domain-based features)

Function to parse `.tblout` files from HMMER outputs and convert them into a structured Pandas DataFrame. The function also allows 
for optional E-value filtering to refine search results based on statistical significance.

#### Features:
- Extracts relevant fields from the `.tblout` format.
- Converts numeric values to appropriate data types.
- Allows filtering by E-value threshold to retain only significant hits.

In [5]:
def parse_tblout(file_path, e_value_threshold=None):
    """
    parse a .tblout file from HMMER output and create a pandas dataframe 
    with optional e-value filtering.

    parameters:
    file_path (str): path to the .tblout file to be parsed.
    e_value_threshold (float, optional): if provided, filters results to include 
                                         only those with e-values below the threshold.

    returns:
    pandas.DataFrame: a dataframe containing parsed data from the .tblout file.
    """
    # define column names
    columns = [
        "target_name", "accession", "tlen", "query_name", "query_accession", "qlen",
        "E-value", "score", "bias", "#", "of", "c-Evalue", "i-Evalue",
        "domain_score", "domain_bias", "hmm_from", "hmm_to", "ali_from", "ali_to",
        "env_from", "env_to", "acc", "description_of_target"
    ]
    
    # read the file
    with open(file_path, "r") as f:
        lines = f.readlines()
    
    # extract data lines (skip header and dashed lines)
    data_lines = lines[3:]  # skip the header and separator lines
    data_lines = [line.strip() for line in data_lines if not line.startswith("#")]
    
    # parse the data into rows
    rows = [line.split() for line in data_lines]
    data_table = pd.DataFrame(rows, columns=columns)
    
    # convert numeric columns to proper types
    numeric_cols = ["E-value", "c-Evalue", "i-Evalue", "score", "domain_score"]
    data_table[numeric_cols] = data_table[numeric_cols].apply(pd.to_numeric, errors="coerce")
    
    # apply E-value filtering if threshold is provided
    if e_value_threshold is not None:
        data_table = data_table[data_table["E-value"] <= e_value_threshold]
    
    return data_table

Generate a binary matrix representing the presence or absence of target sequences across multiple query sequences. The output is formatted 
as a pandas dataframe.

#### features:
- Constructs a binary matrix where rows represent sequences and columns represent targets.
- Ensures values are binary (1 for presence, 0 for absence).
  Fills in missing sequences with zeros for consistency.
  removes columns where all values are zero to reduce redundancy.


In [6]:
def create_binary_matrix(data_table, sequence_count):
    """
    create a binary matrix where sequences are rows and targets are columns, 
    indicating the presence (1) or absence (0) of each target per sequence.

    parameters:
    data_table (pandas.DataFrame): dataframe containing 'query_name' and 'target_name' columns.
    sequence_count (int): total number of sequences to ensure all are represented.

    returns:
    pandas.DataFrame: a binary matrix with sequences as rows and targets as columns.
    """
    # create a binary matrix using crosstab
    binary_matrix = pd.crosstab(data_table["query_name"], data_table["target_name"])
    
    # ensure the values are binary (1/0)
    binary_matrix = binary_matrix.astype(bool).astype(int)
    
    # remove the names for rows and columns
    binary_matrix.index.name = None
    binary_matrix.columns.name = None
    
    # generate the full sequence list (seq1 to seqN)
    full_sequence_list = [f"seq{i}" for i in range(1, sequence_count + 1)]
    
    # reindex the binary matrix to include all sequences, filling missing rows with zeros
    binary_matrix = binary_matrix.reindex(full_sequence_list, fill_value=0)
    binary_matrix.index.name = "sequence"
    
    # remove columns where all values are 0
    binary_matrix = binary_matrix.loc[:, binary_matrix.any(axis=0)]
    
    return binary_matrix

### Generating and saving DNA sequence descriptors

Function to compute various dna descriptors and save each 
one as a separate csv file. 

#### features:
- saves each descriptor as a csv file with an appropriate filename.
- ensures descriptor names are formatted correctly in filenames.
- prints a confirmation message after saving each file.

In [7]:
# function to generate descriptors and save each as a CSV file
def save_all_descriptors(dna, descriptors, output_path):
    """
    generate dna descriptors and save each as a separate csv file.

    parameters:
    dna (object): an object containing dna sequence data and descriptor methods.
    descriptors (list of str): list of descriptor names to compute.
    output_path (str): directory where csv files should be saved.

    returns:
    none
    """
    for descriptor in descriptors:
        # generate the descriptor
        dna.get_descriptor(descriptor)
        
        # construct filename by replacing spaces with underscores
        file_name = f"{descriptor.replace(' ', '_')}_labels.csv"
        
        # save the descriptor as CSV
        dna.to_csv(os.path.join(output_path, file_name), index=True, header=True)
        print(f"Saved {descriptor} to {file_name}")

### Loading sequence based features

In [8]:
def load_data(x_path, y_path):
    """
    loads feature matrix (x) and target variable (y) from csv files, ensuring alignment between the two datasets
    by removing any rows in x that do not have a corresponding entry in y.
    
    parameters:
    x_path (str): path to csv file containing features (index_col=0)
    y_path (str): path to csv file containing target variable ('infection')
    
    returns:
    tuple: x (feature matrix), y (target variable)
    """
    print(f"loading data from {x_path} and {y_path}...")
    y = pd.read_csv(y_path, na_values=['?'])
    y.set_index('sequence', inplace=True)
    y_selected = y['infection']
    
    x = pd.read_csv(x_path, na_values=['?'], index_col=0)
    print(f"original x shape: {x.shape}")
    print(f"original y shape: {y_selected.shape}")
    
    x_selected = x[x.index.isin(y_selected.index)]
    y_selected = y_selected[x_selected.index]
    
    print(f"after alignment - x shape: {x_selected.shape}")
    print(f"after alignment - y shape: {y_selected.shape}")
    
    return x_selected, y_selected

def scale_data(x_train, x_test):
    """
    standardizes the feature values in x_train and x_test using standardscaler.
    this ensures that all features have mean 0 and standard deviation 1, improving model performance.
    
    parameters:
    x_train (dataframe): training feature matrix
    x_test (dataframe): test feature matrix
    
    returns:
    tuple: scaled x_train and x_test as dataframes
    """
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    return (pd.DataFrame(x_train_scaled, index=x_train.index, columns=x_train.columns),
            pd.DataFrame(x_test_scaled, index=x_test.index, columns=x_test.columns))

### Evaluating model

In [9]:
def evaluate_model(x, y, alpha, feature_names, n_splits=10):
    """
    evaluates a machine learning model using stratified k-fold cross-validation.
    performs feature selection with lasso regression (if alpha > 0) and trains a random forest classifier.
    outputs model performance metrics for each fold, including accuracy, precision, recall, and f1 score.
    
    parameters:
    x (dataframe): feature matrix
    y (series): target variable
    alpha (float): regularization strength for lasso feature selection
    feature_names (array-like): names of features
    n_splits (int): number of cross-validation folds
    
    returns:
    dict: dictionary containing evaluation metrics and selected features per fold
    """
    print(f"starting lasso feature selection with alpha={alpha}...")
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_metrics = []
    selected_features_per_fold = []
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(x, y)):
        print(f"processing fold {fold + 1}...")
        x_train, x_test = x.iloc[train_idx], x.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        x_train_scaled, x_test_scaled = scale_data(x_train, x_test)
        
        if alpha == 0:
            x_train_sel, x_test_sel = x_train_scaled, x_test_scaled
            selected_features = list(feature_names)
        else:
            lasso = Lasso(alpha=alpha, random_state=42, max_iter=10000)
            lasso.fit(x_train_scaled, y_train)
            selector = SelectFromModel(lasso, prefit=True)
            x_train_sel = selector.transform(x_train_scaled)
            x_test_sel = selector.transform(x_test_scaled)
            selected_features = list(feature_names[selector.get_support()])
            
            print(f"selected {len(selected_features)} features out of {len(feature_names)} features")
        
        if not selected_features:
            print(f"warning: no features selected in fold {fold+1}")
            continue
        
        selected_features_per_fold.append(selected_features)
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_model.fit(x_train_sel, y_train)
        
        y_pred = rf_model.predict(x_test_sel)
        y_pred_proba = rf_model.predict_proba(x_test_sel)
        
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        fold_metric = {
            'fold': fold + 1,
            'num_features': len(selected_features),
            'accuracy': accuracy_score(y_test, y_pred),
            'balanced_accuracy': (tp/(tp+fn) + tn/(tn+fp))/2 if (tp+fn)*(tn+fp) > 0 else 0,
            'roc_auc': roc_auc_score(y_test, y_pred_proba[:, 1]) * 100,
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0,
            'sensitivity': tp / (tp + fn) if (tp + fn) > 0 else 0,
            'confusion_matrix': [[tn, fp], [fn, tp]],
            'classification_report': classification_report(y_test, y_pred, output_dict=True)
        }
        
        fold_metrics.append(fold_metric)
    
    return {
        'n_splits': n_splits,
        'alpha': alpha,
        'total_samples': len(y),
        'feature_set_size': x.shape[1],
        'selected_features': selected_features_per_fold,
        'fold_metrics': fold_metrics
    }

### Feature selection via LASSO

In [10]:
def run_feature_selection_analysis(feature_files, y_path, output_folder, alphas, n_splits=5):
    """
    runs cross-validation analysis on multiple feature sets with different alpha values.
    
    parameters:
    feature_files (list): list of paths to feature csv files
    y_path (str): path to target variable csv
    output_folder (str): directory where results should be saved
    alphas (list): list of alpha values for lasso feature selection
    n_splits (int): number of cross-validation folds
    
    returns:
    none: saves results as csv files in the output folder
    """
    start_time = time.time()
    print("starting feature selection and model evaluation...")
    
    os.makedirs(output_folder, exist_ok=True)
    all_results = []
    
    for feature_file in feature_files:
        feature_name = os.path.basename(feature_file).replace('.csv', '')
        print(f"\nprocessing {feature_name}")
        
        try:
            # load and process data
            x, y = load_data(feature_file, y_path)
            feature_names = np.array(x.columns)
            
            for alpha in alphas:
                print(f"\nrunning analysis with alpha={alpha}")
                
                result = evaluate_model(x, y, alpha, feature_names, n_splits)
                if result:
                    for fold_result in result['fold_metrics']:
                        tn, fp, fn, tp = fold_result['confusion_matrix'][0][0], fold_result['confusion_matrix'][0][1], \
                                         fold_result['confusion_matrix'][1][0], fold_result['confusion_matrix'][1][1]
                        class_0_metrics = fold_result['classification_report']['0']
                        class_1_metrics = fold_result['classification_report']['1']
                        
                        formatted_result = {
                            'feature_set': feature_name,
                            'fold': fold_result['fold'],
                            'num_features': fold_result['num_features'],
                            'accuracy': round(fold_result['accuracy'] * 100, 4),
                            'balanced_accuracy': round(fold_result['balanced_accuracy'] * 100, 4),
                            'roc_auc': round(fold_result['roc_auc'], 4),
                            'precision': round(fold_result['precision'] * 100, 4),
                            'recall': round(fold_result['recall'] * 100, 4),
                            'f1': round(fold_result['f1'] * 100, 4),
                            'specificity': round(fold_result['specificity'] * 100, 4),
                            'sensitivity': round(fold_result['sensitivity'] * 100, 4),
                            'alpha': alpha,
                            'TP': tp,
                            'FP': fp,
                            'FN': fn,
                            'TN': tn,
                            'class_0_precision': round(class_0_metrics['precision'] * 100, 4),
                            'class_0_recall': round(class_0_metrics['recall'] * 100, 4),
                            'class_0_f1': round(class_0_metrics['f1-score'] * 100, 4),
                            'class_1_precision': round(class_1_metrics['precision'] * 100, 4),
                            'class_1_recall': round(class_1_metrics['recall'] * 100, 4),
                            'class_1_f1': round(class_1_metrics['f1-score'] * 100, 4)
                        }
                        all_results.append(formatted_result)
        
        except Exception as e:
            print(f"error processing {feature_name}: {str(e)}")
            continue
    
    if all_results:
        final_results_df = pd.DataFrame(all_results)
        output_file = os.path.join(output_folder, "combined_results.csv")
        final_results_df.to_csv(output_file, index=False)
        print(f"all results saved to {output_file}")
        
        # generate summary statistics
        summary_df = final_results_df.groupby(['feature_set', 'alpha']).agg({
            'num_features': ['mean', 'std'],
            'accuracy': ['mean', 'std'],
            'balanced_accuracy': ['mean', 'std'],
            'roc_auc': ['mean', 'std'],
            'precision': ['mean', 'std'],
            'recall': ['mean', 'std'],
            'f1': ['mean', 'std'],
            'specificity': ['mean', 'std'],
            'sensitivity': ['mean', 'std'],
            'TP': ['mean', 'std'],
            'FP': ['mean', 'std'],
            'FN': ['mean', 'std'],
            'TN': ['mean', 'std'],
            'class_0_precision': ['mean', 'std'],
            'class_0_recall': ['mean', 'std'],
            'class_0_f1': ['mean', 'std'],
            'class_1_precision': ['mean', 'std'],
            'class_1_recall': ['mean', 'std'],
            'class_1_f1': ['mean', 'std']
        }).reset_index()
        
        # flatten multi-index column names
        summary_df.columns = ['_'.join(col).strip('_') for col in summary_df.columns.values]
        
        # only multiply percentage-based metrics, not raw counts
        percentage_columns = [
            'accuracy_mean', 'accuracy_std', 'balanced_accuracy_mean', 'balanced_accuracy_std',
            'roc_auc_mean', 'roc_auc_std', 'precision_mean', 'precision_std', 'recall_mean', 'recall_std',
            'f1_mean', 'f1_std', 'specificity_mean', 'specificity_std', 'sensitivity_mean', 'sensitivity_std',
            'class_0_precision_mean', 'class_0_precision_std', 'class_0_recall_mean', 'class_0_recall_std',
            'class_0_f1_mean', 'class_0_f1_std', 'class_1_precision_mean', 'class_1_precision_std',
            'class_1_recall_mean', 'class_1_recall_std', 'class_1_f1_mean', 'class_1_f1_std'
        ]
        summary_df[percentage_columns] = summary_df[percentage_columns].round(4)
        
        summary_output_file = os.path.join(output_folder, "summary_results.csv")
        summary_df.to_csv(summary_output_file, index=False)
        print(f"summary results saved to {summary_output_file}")
    
    total_time = time.time() - start_time
    print(f"\nanalysis completed in {total_time:.2f} seconds.")

# run_feature_selection_analysis(
#     feature_files=feature_files,
#     y_path=y_path,
#     output_folder=output_folder,
#     alphas=[0, 0.001, 0.005, 0.01, 0.05, 0.1],  # custom alpha values for feature selection
#     n_splits=10  # number of cross-validation folds
# )

### Hyperparameters grid

In [11]:
def select_features_with_lasso(X, y, alpha=0.01, output_folder=None):
    """
    scales the feature matrix, applies lasso regression for feature selection, and returns the reduced dataset
    containing only features with nonzero coefficients. also saves selected features as a csv file.
    
    parameters:
    X (dataframe): feature matrix
    y (series): target variable
    alpha (float): regularization strength for lasso feature selection
    output_folder (str, optional): directory where selected feature results should be saved
    
    returns:
    tuple: filtered feature matrix, list of selected feature names
    """
    print(f"applying lasso feature selection with alpha={alpha}...")
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    lasso = Lasso(alpha=alpha, random_state=42, max_iter=10000)
    lasso.fit(X_scaled, y)
    
    selected_features = X.columns[np.abs(lasso.coef_) > 0].tolist()
    print(f"lasso selected {len(selected_features)} features out of {X.shape[1]} using alpha = {alpha}")
    
    if output_folder:
        os.makedirs(output_folder, exist_ok=True)
        selected_features_df = pd.DataFrame(selected_features, columns=['selected_features'])
        selected_features_df.to_csv(os.path.join(output_folder, f'selected_features_alpha_{alpha}.csv'), index=False)
        print(f"selected features saved to {output_folder}/selected_features_alpha_{alpha}.csv")
    
    return X[selected_features], selected_features


def run_complete_analysis(X, y, output_folder, lasso_alpha=0.01, cv=5):
    """
    performs feature selection using lasso and hyperparameter tuning for random forest using manual iteration.
    saves the best model parameters, full grid search results per split, and a summary with mean/std per parameter setting.
    
    parameters:
    X (dataframe): feature matrix
    y (series): target variable
    output_folder (str): directory where all results will be saved
    lasso_alpha (float): regularization strength parameter for lasso feature selection
    cv (int): number of folds for stratified cross-validation
    
    returns:
    dict: best model, grid search results, summary metrics, and selected features
    """
    print("starting complete analysis...")
    start_time = time.time()
    
    os.makedirs(output_folder, exist_ok=True)
    
    X_selected, selected_features = select_features_with_lasso(X, y, alpha=lasso_alpha)

    param_grid = {
        'n_estimators': [100, 300, 500],
        'max_depth': [None, 5, 10, 20],  
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2'],
        'bootstrap': [True],
        'class_weight': ['balanced', None]
    }
    
    rf = RandomForestClassifier(random_state=42)
    
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    all_split_metrics = []

    for fold_idx, (train_idx, test_idx) in enumerate(skf.split(X_selected, y), start=1):
        X_train, X_test = X_selected.iloc[train_idx], X_selected.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        for params in [
            {'n_estimators': n, 'max_depth': d, 'min_samples_split': mss, 'min_samples_leaf': msl, 'max_features': mf, 'bootstrap': b, 'class_weight': cw}
            for n in param_grid['n_estimators']
            for d in param_grid['max_depth']
            for mss in param_grid['min_samples_split']
            for msl in param_grid['min_samples_leaf']
            for mf in param_grid['max_features']
            for b in param_grid['bootstrap']
            for cw in param_grid['class_weight']
        ]:
            rf.set_params(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_test)
            y_proba = rf.predict_proba(X_test)[:, 1]

            tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
            classification_rep = classification_report(y_test, y_pred, output_dict=True)
            class_0 = classification_rep.get('0', {})
            class_1 = classification_rep.get('1', {})

            # Create a single column for all hyperparameters
            params_str = str(params)

            split_metrics = {
                'split': fold_idx,
                'params': params_str,  # This column will be used for grouping in the summary
                **params,
                'accuracy': round(accuracy_score(y_test, y_pred) * 100, 4),
                'balanced_accuracy': round(balanced_accuracy_score(y_test, y_pred) * 100, 4),
                'roc_auc': round(roc_auc_score(y_test, y_proba) * 100, 4),
                'precision': round(precision_score(y_test, y_pred, zero_division=0) * 100, 4),
                'recall': round(recall_score(y_test, y_pred, zero_division=0) * 100, 4),
                'f1': round(f1_score(y_test, y_pred, zero_division=0) * 100, 4),
                'specificity': round((tn / (tn + fp)) * 100 if (tn + fp) > 0 else 0, 4),
                'sensitivity': round((tp / (tp + fn)) * 100 if (tp + fn) > 0 else 0, 4),
                'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn,
                'class_0_precision': round(class_0.get('precision', 0) * 100, 4),
                'class_0_recall': round(class_0.get('recall', 0) * 100, 4),
                'class_0_f1': round(class_0.get('f1-score', 0) * 100, 4),
                'class_1_precision': round(class_1.get('precision', 0) * 100, 4),
                'class_1_recall': round(class_1.get('recall', 0) * 100, 4),
                'class_1_f1': round(class_1.get('f1-score', 0) * 100, 4)
            }

            all_split_metrics.append(split_metrics)

    # convert per-split metrics to DataFrame
    split_metrics_df = pd.DataFrame(all_split_metrics)

    # save per-split results
    final_results_file = os.path.join(output_folder, 'grid_search_results_per_split.csv')
    split_metrics_df.to_csv(final_results_file, index=False)
    print(f"grid search results per split saved to {final_results_file}")

    # generate summary (mean and std) per parameter setting
    metric_cols = ['accuracy', 'balanced_accuracy', 'roc_auc', 'precision', 'recall', 'f1', 'specificity', 'sensitivity', 'TP', 'FP', 'FN', 'TN',
                   'class_0_precision', 'class_0_recall', 'class_0_f1', 'class_1_precision', 'class_1_recall', 'class_1_f1']

    if 'params' in split_metrics_df.columns:
        summary_df = split_metrics_df.groupby('params')[metric_cols].agg(['mean', 'std']).reset_index()
        summary_df.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in summary_df.columns]
        summary_df = summary_df.round(4)
    else:
        print("Error: 'params' column missing. Cannot generate summary.")
        summary_df = pd.DataFrame()

    if not summary_df.empty:
        summary_file = os.path.join(output_folder, 'grid_search_summary_per_split.csv')
        summary_df.to_csv(summary_file, index=False)
        print(f"grid search summary per split saved to {summary_file}")
    else:
        print("no summary file generated due to missing 'params' column.")

    total_time = time.time() - start_time
    print(f"\ncomplete analysis finished in {total_time:.2f} seconds.")
    
    return {
        'best_model': rf,
        'grid_search_results': split_metrics_df,
        'grid_search_summary': summary_df if not summary_df.empty else None,
        'selected_features': selected_features
    }
    
#usage

# load and align data
#X, y = load_data(feature_file, label_file)

# run complete analysis
#run_complete_analysis(
#    X=X,
#    y=y,
#    output_folder=output_folder,
#    lasso_alpha=0.01,  # custom alpha for LASSO feature selection
#    cv=10  # number of cross-validation folds
#)

### LOGO

In [12]:
def evaluate_model_with_group_exclusion(X, y, ranks, feature_names, excluded_group, rank_name):
    """
    performs leave-one-group-out (logo) cross-validation by training a model on all groups except one,
    then testing on the excluded group. uses lasso for feature selection followed by a random forest classifier.
    
    parameters:
    X (dataframe): feature matrix with samples as rows and features as columns
    y (series or ndarray): binary target variable (0 or 1)
    ranks (dataframe): dataframe containing a 'rank' column specifying the taxonomic group for each sample
    feature_names (list or index): names of features in X
    excluded_group (str or int): taxonomic group to exclude from training and use for testing
    rank_name (str): taxonomic rank being analyzed (e.g., 'family', 'order', 'genus')
    
    returns:
    dict or none: dictionary with model metrics and selected features, or none if evaluation fails
    """
    train_mask = ranks['rank'] != excluded_group
    test_mask = ranks['rank'] == excluded_group
    
    X_train = X[train_mask]
    y_train = y[train_mask]
    X_test = X[test_mask]
    y_test = y[test_mask]
    
    n_train_pos = sum(y_train == 1)
    n_train_neg = sum(y_train == 0)
    n_test_pos = sum(y_test == 1)
    n_test_neg = sum(y_test == 0)
    
    if len(X_test) < 15:
        print(f"skipping {excluded_group}: insufficient samples ({len(X_test)} < 15)")
        return None
        
    if n_test_pos == 0 or n_test_neg == 0:
        print(f"skipping {excluded_group}: missing class representation (pos: {n_test_pos}, neg: {n_test_neg})")
        return None
    
    print(f"training set: {len(X_train)} samples ({n_train_pos} positive, {n_train_neg} negative)")
    print(f"test set: {len(X_test)} samples ({n_test_pos} positive, {n_test_neg} negative)")
    
    if len(X_train) == 0:
        print("warning: empty train set!")
        return None
    
    if len(np.unique(y_train)) < 2:
        print(f"warning: training set contains only one class ({np.unique(y_train)[0]})")
        return None
    
    X_train_scaled, X_test_scaled = scale_data(X_train, X_test)
    
    alpha = 0.01
    lasso = Lasso(alpha=alpha, random_state=42, max_iter=10000)
    lasso.fit(X_train_scaled, y_train)
    
    feature_importances = np.abs(lasso.coef_)
    selector = SelectFromModel(lasso, prefit=True)
    
    X_train_sel = selector.transform(X_train_scaled)
    X_test_sel = selector.transform(X_test_scaled)
    num_features = X_train_sel.shape[1]
    
    selected_mask = selector.get_support()
    selected_indices = np.where(selected_mask)[0]
    sorted_indices = selected_indices[np.argsort(feature_importances[selected_mask])[::-1]]
    selected_features = list(X_train_scaled.columns[sorted_indices])
    
    if num_features == 0:
        print("no features selected!")
        return None
    
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train_sel, y_train)
    
    y_pred = rf_model.predict(X_test_sel)
    
    # Initialize metrics dictionary with all required columns
    metrics = {
        'Feature_File': '',  # Will be filled later
        'Rank_Type': rank_name,
        'Excluded_Group': excluded_group,
        'Train_Size': len(y_train),
        'Train_Pos': n_train_pos,
        'Train_Neg': n_train_neg,
        'Test_Size': len(y_test),
        'Test_Pos': n_test_pos,
        'Test_Neg': n_test_neg,
        'Num_Features': num_features,
        'num_features': num_features,  # Duplicate for compatibility
        'Selected_Features': selected_features,
        'alpha': alpha
    }
    
    # Calculate basic metrics
    try:
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        metrics.update({
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'sensitivity': tp / (tp + fn) if (tp + fn) > 0 else 0,
            'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0,
            'TP': tp,
            'FP': fp,
            'FN': fn,
            'TN': tn
        })
        metrics['balanced_accuracy'] = (metrics['sensitivity'] + metrics['specificity']) / 2
    except Exception as e:
        print(f"warning: could not compute basic metrics: {str(e)}")
        # Set defaults for failed metrics
        metrics.update({
            'accuracy': np.nan, 'precision': np.nan, 'recall': np.nan, 'f1': np.nan,
            'sensitivity': np.nan, 'specificity': np.nan, 'balanced_accuracy': np.nan,
            'TP': 0, 'FP': 0, 'FN': 0, 'TN': 0
        })
    
    # Calculate per-class metrics
    try:
        report = classification_report(y_test, y_pred, output_dict=True)
        metrics.update({
            'class_0_precision': report['0']['precision'],
            'class_0_recall': report['0']['recall'],
            'class_0_f1': report['0']['f1-score'],
            'class_1_precision': report['1']['precision'],
            'class_1_recall': report['1']['recall'],
            'class_1_f1': report['1']['f1-score'],
        })
    except Exception as e:
        print(f"warning: could not compute per-class metrics: {str(e)}")
        metrics.update({
            'class_0_precision': np.nan, 'class_0_recall': np.nan, 'class_0_f1': np.nan,
            'class_1_precision': np.nan, 'class_1_recall': np.nan, 'class_1_f1': np.nan,
        })
    
    # Calculate probability-based metrics
    try:
        y_pred_proba = rf_model.predict_proba(X_test_sel)
        if y_pred_proba.shape[1] == 2:
            metrics['roc_auc'] = roc_auc_score(y_test, y_pred_proba[:, 1]) * 100
        else:
            metrics['roc_auc'] = np.nan
    except Exception as e:
        print(f"warning: could not compute probability-based metrics: {str(e)}")
        metrics['roc_auc'] = np.nan
    
    return metrics

In [13]:
def run_taxonomic_evaluation(feature_files, y_path, output_folder, rank_paths):
    """
    runs leave-one-group-out (logo) cross-validation across taxonomic ranks.

    for each feature file and taxonomic rank, this function loads feature data and target labels, 
    trains a model excluding each taxonomic group and tests on it, calculates performance metrics, 
    saves results for each taxonomic rank, and combines all results into a single output file.

    parameters:
    feature_files (list of str): paths to feature matrix csv files
    y_path (str): path to csv file with target labels (0 or 1)
    output_folder (str): directory for saving results
    rank_paths (dict): mapping of taxonomic rank names (e.g., 'family', 'order') to csv file paths

    returns:
    tuple: (all_rank_results, combined_results_df)
        - all_rank_results: dict mapping rank names to lists of result dictionaries
        - combined_results_df: dataframe with all combined results, or none if no results
    """
    start_time = time.time()
    print("starting taxonomic evaluation pipeline...")
    
    # Define the standard column order
    ordered_cols = [
        'Feature_File',
        'Rank_Type',
        'Excluded_Group',
        'Train_Size',
        'Train_Pos',
        'Train_Neg',
        'Test_Size',
        'Test_Pos',
        'Test_Neg',
        'Num_Features',
        'Selected_Features',
        'num_features',
        'accuracy',
        'balanced_accuracy',
        'roc_auc',
        'precision',
        'recall',
        'f1',
        'specificity',
        'sensitivity',
        'alpha',
        'TP',
        'FP',
        'FN',
        'TN',
        'class_0_precision',
        'class_0_recall',
        'class_0_f1',
        'class_1_precision',
        'class_1_recall',
        'class_1_f1'
    ]
    
    all_rank_results = {}
    all_results_df = pd.DataFrame([])
    
    for feature_file in feature_files:
        feature_name = os.path.basename(feature_file).replace('.csv', '')
        print(f"\nprocessing {feature_name}")
        
        for rank_name, rank_path in rank_paths.items():
            print(f"\nanalyzing {rank_name} rank...")
            rank_output_folder = os.path.join(output_folder, rank_name.lower())
            os.makedirs(rank_output_folder, exist_ok=True)
            
            try:
                print("loading data...")
                X, y = load_data(feature_file, y_path)
                
                ranks = pd.read_csv(rank_path, index_col=0)
                
                ranks = ranks.loc[X.index]
                
                print(f"loaded {X.shape[0]} sequences with {X.shape[1]} features")
                
                unique_groups = sorted(ranks['rank'].unique())
                print(f"found {len(unique_groups)} unique groups for {rank_name}")
                
                rank_results = []
                
                for excluded_group in unique_groups:
                    print(f"\nevaluating model excluding group {excluded_group}...")
                    
                    result = evaluate_model_with_group_exclusion(
                        X, y, ranks, X.columns, excluded_group, rank_name
                    )
                    
                    if result:
                        result['Feature_File'] = feature_name
                        rank_results.append(result)
                
                if rank_results:
                    all_rank_results[rank_name] = rank_results
                    
                    # Save individual rank results
                    results_df = pd.DataFrame(rank_results)
                    available_cols = [col for col in ordered_cols if col in results_df.columns]
                    results_df = results_df[available_cols]
                    
                    output_file = os.path.join(rank_output_folder, f'{rank_name.lower()}_results.csv')
                    results_df.to_csv(output_file, index=False)
                    print(f"saved {rank_name} results to {output_file}")
                    
                    # Add to combined results
                    all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)
                else:
                    print(f"no results to save for {rank_name}")
                    
            except Exception as e:
                print(f"error processing {rank_name}: {str(e)}")
                continue
    
    # Save combined results
    if not all_results_df.empty:
        available_cols = [col for col in ordered_cols if col in all_results_df.columns]
        all_results_df = all_results_df[available_cols]
        
        combined_output_file = os.path.join(output_folder, 'combined_rank_results.csv')
        all_results_df.to_csv(combined_output_file, index=False)
        print(f"\nsaved combined results to {combined_output_file}")
    else:
        print("no combined results to save")
        all_results_df = None
    
    total_time = time.time() - start_time
    print(f"\nanalysis completed in {total_time:.2f} seconds.")
    
    return all_rank_results, all_results_df

In [14]:
def run_feature_selection_analysis(feature_files, y_path, output_folder, alphas, n_splits=5):
    """
    performs cross-validation analysis on feature sets with different alpha values for lasso.

    loads feature data and target labels, applies lasso-based feature selection, evaluates model 
    performance using cross-validation, and saves results. generates a summary of performance 
    metrics across different feature sets and alpha values.

    parameters:
    feature_files (list of str): paths to feature matrix csv files
    y_path (str): path to csv file with target labels
    output_folder (str): directory for saving results
    alphas (list of float): regularization values for lasso feature selection
    n_splits (int, optional): number of cross-validation folds, default is five

    returns:
    none: saves results and summary as csv files in the output folder
    """
    start_time = time.time()
    print("starting feature selection and model evaluation...")
    
    os.makedirs(output_folder, exist_ok=True)
    all_results = []
    
    # Define the standard column order
    ordered_cols = [
        'feature_set',
        'fold',
        'num_features',
        'accuracy',
        'balanced_accuracy',
        'roc_auc',
        'precision',
        'recall',
        'f1',
        'specificity',
        'sensitivity',
        'alpha',
        'TP',
        'FP',
        'FN',
        'TN',
        'class_0_precision',
        'class_0_recall',
        'class_0_f1',
        'class_1_precision',
        'class_1_recall',
        'class_1_f1'
    ]
    
    for feature_file in feature_files:
        feature_name = os.path.basename(feature_file).replace('.csv', '')
        print(f"\nprocessing {feature_name}")
        
        try:
            # load and process data
            x, y = load_data(feature_file, y_path)
            feature_names = np.array(x.columns)
            
            for alpha in alphas:
                print(f"\nrunning analysis with alpha={alpha}")
                
                result = evaluate_model(x, y, alpha, feature_names, n_splits)
                if result:
                    for fold_result in result['fold_metrics']:
                        tn, fp, fn, tp = fold_result['confusion_matrix'][0][0], fold_result['confusion_matrix'][0][1], \
                                         fold_result['confusion_matrix'][1][0], fold_result['confusion_matrix'][1][1]
                        class_0_metrics = fold_result['classification_report']['0']
                        class_1_metrics = fold_result['classification_report']['1']
                        
                        formatted_result = {
                            'feature_set': feature_name,
                            'fold': fold_result['fold'],
                            'num_features': fold_result['num_features'],
                            'accuracy': round(fold_result['accuracy'] * 100, 4),
                            'balanced_accuracy': round(fold_result['balanced_accuracy'] * 100, 4),
                            'roc_auc': round(fold_result['roc_auc'], 4),
                            'precision': round(fold_result['precision'] * 100, 4),
                            'recall': round(fold_result['recall'] * 100, 4),
                            'f1': round(fold_result['f1'] * 100, 4),
                            'specificity': round(fold_result['specificity'] * 100, 4),
                            'sensitivity': round(fold_result['sensitivity'] * 100, 4),
                            'alpha': alpha,
                            'TP': tp,
                            'FP': fp,
                            'FN': fn,
                            'TN': tn,
                            'class_0_precision': round(class_0_metrics['precision'] * 100, 4),
                            'class_0_recall': round(class_0_metrics['recall'] * 100, 4),
                            'class_0_f1': round(class_0_metrics['f1-score'] * 100, 4),
                            'class_1_precision': round(class_1_metrics['precision'] * 100, 4),
                            'class_1_recall': round(class_1_metrics['recall'] * 100, 4),
                            'class_1_f1': round(class_1_metrics['f1-score'] * 100, 4)
                        }
                        all_results.append(formatted_result)
        
        except Exception as e:
            print(f"error processing {feature_name}: {str(e)}")
            continue
    
    if all_results:
        final_results_df = pd.DataFrame(all_results)
        
        # Ensure columns are in the correct order
        available_cols = [col for col in ordered_cols if col in final_results_df.columns]
        final_results_df = final_results_df[available_cols]
        
        output_file = os.path.join(output_folder, "combined_results.csv")
        final_results_df.to_csv(output_file, index=False)
        print(f"all results saved to {output_file}")
        
        # generate summary statistics
        summary_df = final_results_df.groupby(['feature_set', 'alpha']).agg({
            'num_features': ['mean', 'std'],
            'accuracy': ['mean', 'std'],
            'balanced_accuracy': ['mean', 'std'],
            'roc_auc': ['mean', 'std'],
            'precision': ['mean', 'std'],
            'recall': ['mean', 'std'],
            'f1': ['mean', 'std'],
            'specificity': ['mean', 'std'],
            'sensitivity': ['mean', 'std'],
            'TP': ['mean', 'std'],
            'FP': ['mean', 'std'],
            'FN': ['mean', 'std'],
            'TN': ['mean', 'std'],
            'class_0_precision': ['mean', 'std'],
            'class_0_recall': ['mean', 'std'],
            'class_0_f1': ['mean', 'std'],
            'class_1_precision': ['mean', 'std'],
            'class_1_recall': ['mean', 'std'],
            'class_1_f1': ['mean', 'std']
        }).reset_index()
        
        # flatten multi-index column names
        summary_df.columns = ['_'.join(col).strip('_') for col in summary_df.columns.values]
        
        # only multiply percentage-based metrics, not raw counts
        percentage_columns = [
            'accuracy_mean', 'accuracy_std', 'balanced_accuracy_mean', 'balanced_accuracy_std',
            'roc_auc_mean', 'roc_auc_std', 'precision_mean', 'precision_std', 'recall_mean', 'recall_std',
            'f1_mean', 'f1_std', 'specificity_mean', 'specificity_std', 'sensitivity_mean', 'sensitivity_std',
            'class_0_precision_mean', 'class_0_precision_std', 'class_0_recall_mean', 'class_0_recall_std',
            'class_0_f1_mean', 'class_0_f1_std', 'class_1_precision_mean', 'class_1_precision_std',
            'class_1_recall_mean', 'class_1_recall_std', 'class_1_f1_mean', 'class_1_f1_std'
        ]
        summary_df[percentage_columns] = summary_df[percentage_columns].round(4)
        
        summary_output_file = os.path.join(output_folder, "summary_results.csv")
        summary_df.to_csv(summary_output_file, index=False)
        print(f"summary results saved to {summary_output_file}")
    else:
        print("no results to save")
    
    total_time = time.time() - start_time
    print(f"\nanalysis completed in {total_time:.2f} seconds.")
    
    return None 

In [15]:
import pandas as pd

def generate_metrics_summary(input_file, output_file):
    """
    scales and summarizes performance metrics by feature_file and rank_type, calculating mean and standard deviation.
    excludes entries where excluded_group is "notassigned" and saves results to a csv file.
    
    parameters:
    input_file (str): path to the input csv/tsv file containing the metrics data
    output_file (str): path where the output csv will be saved
    
    returns:
    dataframe: summary statistics with percentage values rounded appropriately
    """
    # read the input data - using comma separator (not tab)
    df = pd.read_csv(input_file)  # default is comma separator
    
    # print column names to verify
    print(f"available columns in the dataset ({len(df.columns)} columns):")
    print(df.columns.tolist())
    
    # convert relevant metrics to percentages (multiply by 100)
    percentage_columns = [
        'accuracy', 'balanced_accuracy', 'roc_auc', 'precision', 'recall', 'f1',
        'specificity', 'sensitivity', 'class_0_precision', 'class_0_recall', 
        'class_0_f1', 'class_1_precision', 'class_1_recall', 'class_1_f1'
    ]
    
    for col in percentage_columns:
        if col in df.columns:
            df[col] = df[col] * 100
    
    # filter out rows where excluded_group is "notassigned"
    filtered_df = df[df['Excluded_Group'] != 'NotAssigned']
    
    print(f"original data: {len(df)} rows")
    print(f"after filtering out notassigned: {len(filtered_df)} rows")
    
    # group by feature_file and rank_type
    summary_df = filtered_df.groupby(['Feature_File', 'Rank_Type']).agg({
        'Train_Size': ['mean', 'std'],
        'Train_Pos': ['mean', 'std'],
        'Train_Neg': ['mean', 'std'],
        'Test_Size': ['mean', 'std'],
        'Test_Pos': ['mean', 'std'],
        'Test_Neg': ['mean', 'std'],
        'Num_Features': ['mean', 'std'],
        'accuracy': ['mean', 'std'],
        'balanced_accuracy': ['mean', 'std'],
        'roc_auc': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'specificity': ['mean', 'std'],
        'sensitivity': ['mean', 'std'],
        'alpha': ['mean', 'std'],
        'TP': ['mean', 'std'],
        'FP': ['mean', 'std'],
        'FN': ['mean', 'std'],
        'TN': ['mean', 'std'],
        'class_0_precision': ['mean', 'std'],
        'class_0_recall': ['mean', 'std'],
        'class_0_f1': ['mean', 'std'],
        'class_1_precision': ['mean', 'std'],
        'class_1_recall': ['mean', 'std'],
        'class_1_f1': ['mean', 'std']
    }).reset_index()
    
    # flatten multi-index column names
    summary_df.columns = ['_'.join(col).strip('_') for col in summary_df.columns.values]
    
    # add sample count for each group
    group_counts = filtered_df.groupby(['Feature_File', 'Rank_Type']).size().reset_index(name='Sample_Count')
    summary_df = summary_df.merge(group_counts, on=['Feature_File', 'Rank_Type'])
    
    # format all numerical values to 4 decimal places
    for col in summary_df.columns:
        if summary_df[col].dtype in ['float64', 'float32']:
            summary_df[col] = summary_df[col].round(4)
    
    # save to csv
    summary_df.to_csv(output_file, index=False)
    
    print(f"generated summary with {len(summary_df)} rows")
    print(f"results saved to {output_file}")
    print(f"all metrics are now displayed with a maximum of 4 decimal places")
    
    return summary_df

### Feature importance via SHAP

In [17]:
def train_random_forest(X, y):
    """
    train a random forest classifier.
    
    parameters:
    X (pandas.DataFrame): feature matrix
    y (pandas.Series): target variable
    
    returns:
    tuple: trained model, test set (X_test, y_test)
    """
    # split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # train a random forest model
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # evaluate the model
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {acc:.4f}")

    return model, X_test, y_test


def perform_shap_analysis(model, X_test, output_folder):
    """
    perform SHAP analysis on the trained model and save results.
    
    parameters:
    model (RandomForestClassifier): trained model
    X_test (pandas.DataFrame): test feature matrix
    output_folder (str): folder to save SHAP results
    
    returns:
    None
    """
    os.makedirs(output_folder, exist_ok=True)

    # ensure X_test has the correct shape
    assert X_test.shape[1] == model.n_features_in_, "feature mismatch between X_test and trained model!"

    # initialize SHAP Explainer with interventional perturbation
    explainer = shap.TreeExplainer(model, feature_perturbation="interventional")

    # compute SHAP values with additivity check disabled
    shap_values = explainer.shap_values(X_test, check_additivity=False)

    # handle multi-class case 
    print(f"original SHAP values type: {type(shap_values)}")
    
    if isinstance(shap_values, list):
        print(f"SHAP values is a list with {len(shap_values)} elements")
        for i, sv in enumerate(shap_values):
            print(f"shape of shap_values[{i}]: {sv.shape}")
        
        # save SHAP values for both classes
        # class 1 (positive class) SHAP values
        shap_df = pd.DataFrame(shap_values[1], columns=X_test.columns)
        shap_df.to_csv(f"{output_folder}/shap_values_class1.csv", index=False)
        print(f"SHAP values for class 1 saved to {output_folder}/shap_values_class1.csv")
        
        # class 0 SHAP values
        shap_df_class0 = pd.DataFrame(shap_values[0], columns=X_test.columns)
        shap_df_class0.to_csv(f"{output_folder}/shap_values_class0.csv", index=False)
        print(f"SHAP values for class 0 saved to {output_folder}/shap_values_class0.csv")
    else:
        # if not a list, it might be a single array or have a different structure
        print(f"SHAP values shape: {shap_values.shape}")
        
        # try to determine the correct dimensionality
        if len(shap_values.shape) == 3:  # 3D array
            print("detected 3D SHAP values array")
            # Assuming last dimension is for classes, get class 1
            shap_values_for_plot = shap_values[:, :, 1]
            print(f"reshaped for plotting: {shap_values_for_plot.shape}")
        else:
            shap_values_for_plot = shap_values
            
        # save the appropriate SHAP values
        try:
            shap_df = pd.DataFrame(shap_values_for_plot, columns=X_test.columns)
            shap_df.to_csv(f"{output_folder}/shap_values.csv", index=False)
            print(f"SHAP values saved to {output_folder}/shap_values.csv")
        except Exception as e:
            print(f"error saving SHAP values as DataFrame: {e}")
            # Fallback - save as numpy array
            np.save(f"{output_folder}/shap_values.npy", shap_values)
            print(f"saved raw SHAP values as numpy array instead")

    # Create a BLUE TO GREEN colormap (as requested)
    from matplotlib.colors import LinearSegmentedColormap
    import matplotlib.colors as mcolors
    
    # Convert hex to RGB for matplotlib
    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16)/255 for i in (0, 2, 4))
    
    # INVERTED gradient with SOFTER colors: emerald green to dark teal (as requested)
    soft_emerald = hex_to_rgb('#48cfe0')  # Softer emerald green
    soft_teal = hex_to_rgb('#649e2b')     # Softer dark teal
    #649e2b
    # Create the inverted & softer gradient (emerald to teal)
    custom_cmap = LinearSegmentedColormap.from_list('SoftEmeraldToTeal', [soft_emerald, soft_teal], N=100)

    try:
        # First summary plot - make it TALLER than wider (as requested)
        # Using dimensions that are more vertical like original SHAP function
        plt.figure(figsize=(12, 25))  # TALLER than wider like the original SHAP
        if isinstance(shap_values, list) and len(shap_values) >= 2:
            print("plotting SHAP summary for top 40 features")
            # get feature importance based on mean absolute SHAP values
            feature_importance = np.abs(shap_values[1]).mean(0)
            top_indices = np.argsort(-feature_importance)[:40]  # get indices of top 40 features
            
            # get a dataframe with only the top 40 features
            X_test_top40 = X_test.iloc[:, top_indices]
            shap_values_top40 = shap_values[1][:, top_indices]
            
            # plot with only the top 40 features using teal-to-emerald colormap
            shap.summary_plot(shap_values_top40, X_test_top40, show=False, max_display=40, 
                             plot_size=(12, 25), cmap=custom_cmap)
        else:
            # handle the case for different format of shap_values
            if 'shap_values_for_plot' in locals():
                feature_importance = np.abs(shap_values_for_plot).mean(0)
            else:
                feature_importance = np.abs(shap_values).mean(0)
                
            top_indices = np.argsort(-feature_importance)[:40]
            X_test_top40 = X_test.iloc[:, top_indices]
            
            if 'shap_values_for_plot' in locals():
                shap_values_top40 = shap_values_for_plot[:, top_indices]
            else:
                shap_values_top40 = shap_values[:, top_indices]
                
            shap.summary_plot(shap_values_top40, X_test_top40, show=False, max_display=40,
                             plot_size=(12, 25), cmap=custom_cmap)
            
        plt.savefig(f"{output_folder}/shap_summary_plot_top40.svg", bbox_inches='tight', format='svg')
        plt.close()
        print(f"SHAP summary plot with top 40 features saved to {output_folder}/shap_summary_plot_top40.svg")
        
        # Second summary plot - 20 features, also TALLER than wider
        plt.figure(figsize=(10, 15))  # TALLER than wider
        if isinstance(shap_values, list) and len(shap_values) >= 2:
            print("plotting SHAP summary with 20 features")
            # Get top 20 features for second plot
            top20_indices = np.argsort(-feature_importance)[:20]
            X_test_top20 = X_test.iloc[:, top20_indices]
            shap_values_top20 = shap_values[1][:, top20_indices]
            
            shap.summary_plot(shap_values_top20, X_test_top20, show=False, max_display=20, 
                              plot_size=(10, 15), cmap=custom_cmap)
        else:
            # Get top 20 features for second plot
            top20_indices = np.argsort(-feature_importance)[:20]
            X_test_top20 = X_test.iloc[:, top20_indices]
            
            if 'shap_values_for_plot' in locals():
                shap_values_top20 = shap_values_for_plot[:, top20_indices]
            else:
                shap_values_top20 = shap_values[:, top20_indices]
                
            shap.summary_plot(shap_values_top20, X_test_top20, show=False, max_display=20, 
                             plot_size=(10, 15), cmap=custom_cmap)
                
        plt.savefig(f"{output_folder}/shap_summary_plot_top20.svg", bbox_inches='tight', format='svg')
        plt.close()
        print(f"SHAP summary plot with 20 features saved to {output_folder}/shap_summary_plot_top20.svg")
    except Exception as e:
        print(f"error generating summary plots: {e}")
        import traceback
        traceback.print_exc()

    # SHAP bar plot with 20 features - using ACTUAL TEAL COLOR as specifically requested
    try:
        plt.figure(figsize=(10, 12))  # Taller than wider for bar plot too
        if isinstance(shap_values, list) and len(shap_values) >= 2:
            print("plotting SHAP bar plot with 20 features")
            # Get top 20 features for bar plot
            feature_importance = np.abs(shap_values[1]).mean(0)
            top20_indices = np.argsort(-feature_importance)[:20]
            X_test_top20 = X_test.iloc[:, top20_indices]
            shap_values_top20 = shap_values[1][:, top20_indices]
            
            # Use SOFTER TEAL color for bar plot
            shap.summary_plot(shap_values_top20, X_test_top20, plot_type="bar", show=False, max_display=20,
                              color='#66B2B2')  # Softer teal color
        else:
            # Get top 20 features
            if 'shap_values_for_plot' in locals():
                feature_importance = np.abs(shap_values_for_plot).mean(0)
            else:
                feature_importance = np.abs(shap_values).mean(0)
                
            top20_indices = np.argsort(-feature_importance)[:20]
            X_test_top20 = X_test.iloc[:, top20_indices]
            
            if 'shap_values_for_plot' in locals():
                shap_values_top20 = shap_values_for_plot[:, top20_indices]
            else:
                shap_values_top20 = shap_values[:, top20_indices]
                
            # Use SOFTER TEAL color for bar plot
            shap.summary_plot(shap_values_top20, X_test_top20, plot_type="bar", 
                              show=False, max_display=20, color='#66B2B2')  # Softer teal color
                
        plt.savefig(f"{output_folder}/shap_bar_plot.svg", bbox_inches='tight', format='svg')
        plt.close()
        print(f"SHAP bar plot with 20 features saved to {output_folder}/shap_bar_plot.svg")
    except Exception as e:
        print(f"error generating bar plot: {e}")
    
    # generate individual feature plots for top features
    try:
        # Get feature importance - using mean absolute SHAP value
        if isinstance(shap_values, list) and len(shap_values) >= 2:
            # For binary classification with list of arrays
            class1_values = shap_values[1]
            mean_abs_shap = np.abs(class1_values).mean(axis=0)
        elif len(shap_values.shape) == 3:
            # For 3D array, use the class 1 values (index 1 on last dimension)
            class1_values = shap_values[:, :, 1]
            mean_abs_shap = np.abs(class1_values).mean(axis=0)
        else:
            mean_abs_shap = np.abs(shap_values).mean(axis=0)
        
        # explicitly create feature importance with column names
        feature_imp = pd.Series(mean_abs_shap, index=X_test.columns)
        print(f"feature importance shape: {feature_imp.shape}")
        
        # get top 40 features by importance 
        top40_features = feature_imp.sort_values(ascending=False).head(40)
        print(f"top 40 features: {top40_features.index.tolist()}")
        
        # create directory for feature plots
        feature_plots_dir = f"{output_folder}/feature_plots"
        os.makedirs(feature_plots_dir, exist_ok=True)
        
        # save feature importance to CSV
        feature_imp.sort_values(ascending=False).to_csv(
            f"{output_folder}/feature_importance.csv")
        print(f"feature importance saved to {output_folder}/feature_importance.csv")
        
        # generate plots for top features using their names directly
        for i, (feature_name, importance) in enumerate(top40_features.items()):
            print(f"generating plot for feature: {feature_name} (importance: {importance:.4f})")
            
            # get the feature index from the column name
            feature_idx = X_test.columns.get_loc(feature_name)
            
            plt.figure(figsize=(8, 6))
            try:
                if isinstance(shap_values, list) and len(shap_values) >= 2:
                    # Use the blue-green colormap for dependence plots
                    shap.dependence_plot(
                        feature_idx, 
                        shap_values[1], 
                        X_test, 
                        feature_names=X_test.columns.tolist(),
                        show=False,
                        dot_size=10,
                        alpha=0.8,
                        cmap=custom_cmap
                    )
                elif len(shap_values.shape) == 3:
                    shap_class1 = shap_values[:, :, 1]
                    shap.dependence_plot(
                        feature_idx, 
                        shap_class1, 
                        X_test, 
                        feature_names=X_test.columns.tolist(),
                        show=False,
                        dot_size=10,
                        alpha=0.8,
                        cmap=custom_cmap
                    )
                
                plt.title(f"SHAP dependence plot - {feature_name}")
                safe_name = str(feature_name).replace('/', '_').replace('\\', '_').replace(':', '_')
                plt.savefig(f"{feature_plots_dir}/feature_{i+1}_{safe_name}.svg", 
                           bbox_inches='tight', format='svg')
                print(f"  saved plot for feature {feature_name}")
            except Exception as e:
                print(f"  error plotting feature {feature_name}: {e}")
            finally:
                plt.close()
                
        print(f"generated individual feature plots for top 40 features in {feature_plots_dir}")
    except Exception as e:
        print(f"error generating feature plots: {e}")
        import traceback
        traceback.print_exc()

# Sequence processing and feature extraction

### nucleotide features

In [17]:
# paths
input_fasta_nuc = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/sequences/nuc_sequences.fasta"
output_fasta_nuc = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/sequences/nuc_sequences_with_N.fasta"

# replace ambiguous bases in the input fasta file and save the cleaned sequences to the output file
total_amb, total_bases = clean_sequences_with_N(input_fasta_nuc, output_fasta_nuc)
print(f"Total ambiguous bases replaced: {total_amb}/{total_bases} ({(total_amb/total_bases)*100:.2f}%)")

# extract dna sequence descriptors using ifeatureomegacli
dna = iFeatureOmegaCLI.iDNA("/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/sequences/nuc_sequences_with_N.fasta")
dna.import_parameters('/parameters/DNA_parameters_setting.json')

# list of dna descriptors to calculate
descriptors = ["NAC", "Z_curve_9bit", "Z_curve_12bit", "Z_curve_48bit", "Z_curve_144bit", "CKSNAP type 1 ", "Kmer type 1 "]

# path where to save the files
output_path_descriptors = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/"
save_all_descriptors(rna, descriptors, output_path_descriptors)

Total ambiguous bases replaced: 29359/38391088 (0.08%)
Saved Z_curve_9bit to Z_curve_9bit_labels.csv
Saved Z_curve_12bit to Z_curve_12bit_labels.csv
Saved Z_curve_48bit to Z_curve_48bit_labels.csv


### nucleotide features performance

In [7]:
# Define paths
feature_files = [
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/kmer_3.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/kmer_4.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/kmer_5.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/kmer_6.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/NAC.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/CKSNAP.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/Z_curve_9bit.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/Z_curve_12bit.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/Z_curve_48bit.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/Z_curve_144bit.csv"
]
y_path = '/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv'  # CSV with a 'target' column containing 0s and 1s
output_folder = '/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results'

run_feature_selection_analysis(
    feature_files=feature_files,
    y_path=y_path,
    output_folder=output_folder,
    alphas=[0, 0.001, 0.005, 0.01, 0.05, 0.1],  # custom alpha values for feature selection
    n_splits=10  # number of cross-validation folds
)

starting feature selection and model evaluation...

processing kmer_3
loading data from /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/kmer_3.csv and /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv...
original x shape: (3764, 64)
original y shape: (3764,)
after alignment - x shape: (3764, 64)
after alignment - y shape: (3764,)

running analysis with alpha=0
starting lasso feature selection with alpha=0...
processing fold 1...
processing fold 2...
processing fold 3...
processing fold 4...
processing fold 5...
processing fold 6...
processing fold 7...
processing fold 8...
processing fold 9...
processing fold 10...

processing kmer_4
loading data from /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/kmer_4.csv and /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv...
original x shape: (3764, 256)
original y shape: (3764,)
af

In [38]:
## merge selected nucleotide features 

# list of CSV file paths
csv_files = [
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/kmer_3.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/CKSNAP.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/Z_curve_48bit.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features/vog_table.csv",
]

output_file = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/mix/nuc_vog_features.csv"

# read all CSVs and merge them on the index column
nuc_features = [pd.read_csv(file, index_col=0) for file in csv_files]

# concatenating all dataframes along columns (axis=1) based on index
merged_nuc_features = pd.concat(nuc_features, axis=1)

# save the DataFrame to CSV
merged_nuc_features.to_csv(output_file)

### Protein features

After running `hmmscan` to scan protein sequences against a Hidden Markov Model (HMM) database and getting the results

In [14]:
# parse vog and vfam txt
vog_table_self_trans = parse_tblout("/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/phmms/vog_matches_domtblout.txt", e_value_threshold=0.01)
vfam_table_self_trans = parse_tblout("/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/phmms/vfam_matches_domtblout.txt", e_value_threshold=0.01)

# remove "_frameX" from query_name
vog_table_self_trans['query_name'] = vog_table_self_trans['query_name'].str.replace(r'_frame\d+', '', regex=True)
# drop duplicates where target_name and query_name are the same, keeping one randomly
vog_table_self_trans = vog_table_self_trans.drop_duplicates(subset=['target_name', 'query_name'], keep='first')

# remove "_frameX" from query_name
vfam_table_self_trans['query_name'] = vfam_table_self_trans['query_name'].str.replace(r'_frame\d+', '', regex=True)
# drop duplicates where target_name and query_name are the same, keeping one randomly
vfam_table_self_trans = vfam_table_self_trans.drop_duplicates(subset=['target_name', 'query_name'], keep='first')

# create binary matrices ## automatice later
vog_trans_binary_matrix = create_binary_matrix(vog_table_self_trans, sequence_count=3764)
vfam_trans_binary_matrix = create_binary_matrix(vfam_table_self_trans, sequence_count=3764)

# concatenate vog and vfam
allphmms_trans_binary_matrix = pd.concat([vog_trans_binary_matrix, vfam_trans_binary_matrix], axis=1)

# replace NaN values with 0 to ensure binary data
allphmms_trans_binary_matrix = allphmms_trans_binary_matrix.fillna(0).astype(int)

# save tables to CSV file
vog_trans_binary_matrix.to_csv(
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features/vog_table.csv",
    index=True)
vfam_trans_binary_matrix.to_csv(
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features/vfam_table.csv",
    index=True)
allphmms_trans_binary_matrix.to_csv(
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features/allphmms_table.csv",
    index=True)

# display binary matrices
print("\nVOG Binary Matrix:")
print(vog_trans_binary_matrix.shape)

print("\nVFAM Binary Matrix:")
print(vfam_trans_binary_matrix.shape)

print("\nPHMMs Binary Matrix:")
print(allphmms_trans_binary_matrix.shape)


VOG Binary Matrix:
(3764, 4668)

VFAM Binary Matrix:
(3764, 2651)

PHMMs Binary Matrix:
(3764, 7319)


# Machine learning model

## Feature selection via LASSO regularisation

In [ ]:
# define paths
feature_files = [
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/kmer_3.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/CKSNAP.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/Z_curve_48bit.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/nuc_features/merged_nuc_features.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features/vfam_table.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features/vfam_table.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features/vog_table.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features/allphmms_table.csv",
    "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/mix/all_features.csv"
]
y_path = '/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv'  # CSV with a 'target' column containing 0s and 1s
output_folder = '/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/all_features'

run_feature_selection_analysis(
    feature_files=feature_files,
    y_path=y_path,
    output_folder=output_folder,
    alphas=[0, 0.001, 0.005, 0.01, 0.05, 0.1],  # custom alpha values for feature selection
    n_splits=10  # number of cross-validation folds
)

## Searching best hyperparameters

In [50]:
# assign paths
x_path = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/mix/nuc_vog_features.csv"
y_path = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv"
output_folder = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/hyperparameters/nuc_vog_features"

# load and prepare data
X, y = load_data(x_path, y_path)

# run hyperparameter grid analysis
results = run_complete_analysis(X, y, output_folder, lasso_alpha=0.01, cv=10)

loading data from /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/mix/nuc_vog_features.csv and /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv...
original x shape: (3748, 4844)
original y shape: (3748,)
after alignment - x shape: (3748, 4844)
after alignment - y shape: (3748,)
starting complete analysis...
applying lasso feature selection with alpha=0.01...
lasso selected 170 features out of 4844 using alpha = 0.01
grid search results per split saved to /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/hyperparameters/nuc_vog_features/grid_search_results_per_split.csv
grid search summary per split saved to /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/hyperparameters/nuc_vog_features/grid_search_summary_per_split.csv

complete analysis finished in 2474.14 seconds.


## Saving best feature dataset

In [45]:
# define paths
x_path = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features/vog_table.csv"
y_path = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv"
output_folder = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/selected_dataset/vog_features_reduced.csv"

# load and prepare data
X, y = load_data(x_path, y_path)

# run model evaluation with lasso alpha=0.01
feature_names = np.array(X.columns)
results = evaluate_model(X, y, alpha=0.01, feature_names=feature_names)

# extract selected features across folds
all_selected_features = [feature for fold_features in results['selected_features'] for feature in fold_features]
feature_counts = Counter(all_selected_features)

# select features that appear in at least 80% of the folds
n_folds = results['n_splits']
threshold = int(n_folds * 0.8)  # 80% of folds
consensus_features = [feature for feature, count in feature_counts.items() if count >= threshold]

print(f"selected {len(consensus_features)} consensus features out of {X.shape[1]} original features")

# create and save the feature matrix with only selected features
X_selected = X[consensus_features]
X_selected.to_csv(output_folder, index=True)

loading data from /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features/vog_table.csv and /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv...
original x shape: (3748, 4668)
original y shape: (3748,)
after alignment - x shape: (3748, 4668)
after alignment - y shape: (3748,)
starting lasso feature selection with alpha=0.01...
processing fold 1...
selected 154 features out of 4668 features
processing fold 2...
selected 162 features out of 4668 features
processing fold 3...
selected 144 features out of 4668 features
processing fold 4...
selected 164 features out of 4668 features
processing fold 5...
selected 161 features out of 4668 features
processing fold 6...
selected 150 features out of 4668 features
processing fold 7...
selected 167 features out of 4668 features
processing fold 8...
selected 159 features out of 4668 features
processing fold 9...
selected 159 features out of 4668 features
processing fold 10..

## Feature importance analysis

In [18]:
# load and preprocess data
x_path = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/selected_dataset/nuc_vog_features_reduced.csv"
y_path = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv"
output_folder = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/shap/"

X, y = load_data(x_path, y_path)

# train model
model, X_test, y_test = train_random_forest(X, y)

# perform SHAP analysis
perform_shap_analysis(model, X_test, output_folder)

loading data from /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/selected_dataset/nuc_vog_features_reduced.csv and /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv...
original x shape: (3748, 147)
original y shape: (3748,)
after alignment - x shape: (3748, 147)
after alignment - y shape: (3748,)
Model Accuracy: 0.9760
original SHAP values type: <class 'numpy.ndarray'>
SHAP values shape: (750, 147, 2)
detected 3D SHAP values array
reshaped for plotting: (750, 147)
SHAP values saved to /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/shap//shap_values.csv
SHAP summary plot with top 40 features saved to /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/shap//shap_summary_plot_top40.svg
SHAP summary plot with 20 features saved to /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/shap//shap_summary_plot_top20.svg
SHAP bar pl

## LOGO

In [47]:
# define paths
feature_files = ["/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/selected_dataset/vog_features_reduced.csv"]
y_path = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv"
output_folder = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/logo/vog_features"
rank_paths = {
    'Phylum': "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y_phylum.csv",
    'Class': "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y_class.csv",
    'Order': "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y_order.csv",
    'Family': "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y_family.csv",
    'Genus': "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y_genus.csv"
}

# run logo
run_taxonomic_evaluation(feature_files, y_path, output_folder, rank_paths)

starting taxonomic evaluation pipeline...

processing vog_features_reduced

analyzing Phylum rank...
loading data...
loading data from /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/selected_dataset/vog_features_reduced.csv and /Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv...
original x shape: (3748, 141)
original y shape: (3748,)
after alignment - x shape: (3748, 141)
after alignment - y shape: (3748,)
loaded 3748 sequences with 141 features
found 7 unique groups for Phylum

evaluating model excluding group Artverviricota...
training set: 3664 samples (1116 positive, 2548 negative)
test set: 84 samples (1 positive, 83 negative)

evaluating model excluding group Duplornaviricota...
training set: 3509 samples (1095 positive, 2414 negative)
test set: 239 samples (22 positive, 217 negative)

evaluating model excluding group Kitrinoviricota...
training set: 2950 samples (600 positive, 2350 negative)
test set: 798 sa

({'Phylum': [{'Feature_File': 'vog_features_reduced',
    'Rank_Type': 'Phylum',
    'Excluded_Group': 'Artverviricota',
    'Train_Size': 3664,
    'Train_Pos': 1116,
    'Train_Neg': 2548,
    'Test_Size': 84,
    'Test_Pos': 1,
    'Test_Neg': 83,
    'Num_Features': 140,
    'num_features': 140,
    'Selected_Features': ['VOG53050',
     'VOG12513',
     'VOG07215',
     'VOG10740',
     'VOG70232',
     'VOG52932',
     'VOG62629',
     'VOG24940',
     'VOG67308',
     'VOG71803',
     'VOG05362',
     'VOG70661',
     'VOG21255',
     'VOG13092',
     'VOG48747',
     'VOG72932',
     'VOG57786',
     'VOG53260',
     'VOG31849',
     'VOG66017',
     'VOG72217',
     'VOG60592',
     'VOG05356',
     'VOG38427',
     'VOG63028',
     'VOG68409',
     'VOG35661',
     'VOG29157',
     'VOG72071',
     'VOG12481',
     'VOG69013',
     'VOG64064',
     'VOG14318',
     'VOG10925',
     'VOG24439',
     'VOG01966',
     'VOG05328',
     'VOG25616',
     'VOG13223',
     'VOG72298'

In [49]:
# Check if input file exists
input_rank_stats = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/logo/nuc_vog_features/combined_rank_results.csv"
output_rank_summary = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/logo/nuc_vog_features/combined_rank_summary.csv"

print(f"Checking if input file exists: {os.path.exists(input_rank_stats)}")

# Try different ways to read the file
try:
    # First try with default comma separator
    df = pd.read_csv(input_rank_stats)
    print("Successfully read file with comma separator")
    print(f"Columns: {df.columns.tolist()}")
except Exception as e:
    print(f"Error with comma separator: {e}")
    try:
        # Then try with tab separator
        df = pd.read_csv(input_rank_stats, sep='\t')
        print("Successfully read file with tab separator")
        print(f"Columns: {df.columns.tolist()}")
    except Exception as e:
        print(f"Error with tab separator: {e}")
        try:
            # Try with auto-detection of separator
            df = pd.read_csv(input_rank_stats, sep=None, engine='python')
            print("Successfully read file with auto-detected separator")
            print(f"Columns: {df.columns.tolist()}")
        except Exception as e:
            print(f"Error with auto-detection: {e}")

# If we get to this point and df is defined, proceed with your function
if 'df' in locals():
    # Now call your function
    generate_metrics_summary(input_rank_stats, output_rank_summary)
else:
    print("Could not read the input file")

Checking if input file exists: True
Successfully read file with comma separator
Columns: ['Feature_File', 'Rank_Type', 'Excluded_Group', 'Train_Size', 'Train_Pos', 'Train_Neg', 'Test_Size', 'Test_Pos', 'Test_Neg', 'Num_Features', 'Selected_Features', 'num_features', 'accuracy', 'balanced_accuracy', 'roc_auc', 'precision', 'recall', 'f1', 'specificity', 'sensitivity', 'alpha', 'TP', 'FP', 'FN', 'TN', 'class_0_precision', 'class_0_recall', 'class_0_f1', 'class_1_precision', 'class_1_recall', 'class_1_f1']
available columns in the dataset (31 columns):
['Feature_File', 'Rank_Type', 'Excluded_Group', 'Train_Size', 'Train_Pos', 'Train_Neg', 'Test_Size', 'Test_Pos', 'Test_Neg', 'Num_Features', 'Selected_Features', 'num_features', 'accuracy', 'balanced_accuracy', 'roc_auc', 'precision', 'recall', 'f1', 'specificity', 'sensitivity', 'alpha', 'TP', 'FP', 'FN', 'TN', 'class_0_precision', 'class_0_recall', 'class_0_f1', 'class_1_precision', 'class_1_recall', 'class_1_f1']
original data: 66 rows
a

## GO terms

In [ ]:
# load the dataset
features = pd.read_csv('/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/selected_dataset/nuc_vog_selected_features.csv')

# extract features that start with "vfam"
vfam_features = features[features['selected_features'].str.startswith('VFAM')]

# extract features that start with "vog"
vog_features = features[features['selected_features'].str.startswith('VOG')]

# get the lists of feature names
vfam_names = vfam_features['selected_features'].tolist()
vog_names = vog_features['selected_features'].tolist()

print("\nvfam feature names:", vfam_names)
print("vog feature names:", vog_names)

# process vfam and vog annotations to create unified_df
# initialize empty dataframes
vfam_df = pd.DataFrame(columns=["Group", "Protein Name", "UniProt ID", "Source"])
vog_df = pd.DataFrame(columns=["Group", "Protein Name", "UniProt ID", "Source"])

# process vfam data if available
if len(vfam_names) > 0:
    try:
        # read the vfam annotations df
        vfam_annotations = pd.read_csv('/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/phmms/vfam.annotations.tsv', sep='\t')
        
        # filter the vfam_annotations 
        filtered_vfam_annotations = vfam_annotations[vfam_annotations['#GroupName'].isin(vfam_names)].copy()
        
        if not filtered_vfam_annotations.empty:
            # extract both the primary accession number and protein name
            vfam_protein_data = filtered_vfam_annotations.apply(
                lambda row: (
                    row['#GroupName'],  # vfam group
                    row['ConsensusFunctionalDescription'].split('|')[1] if '|' in row['ConsensusFunctionalDescription'] and len(row['ConsensusFunctionalDescription'].split('|')) > 1 else 'not found',  # uniprot id
                    row['ConsensusFunctionalDescription'].split('|')[2].split(' ')[0] if '|' in row['ConsensusFunctionalDescription'] and len(row['ConsensusFunctionalDescription'].split('|')) > 2 else 'not found'  # protein name
                ),
                axis=1
            ).values.tolist()
            
            if vfam_protein_data:
                # convert list of tuples into three separate lists
                vfam_groups, vfam_accessions, vfam_protein_names = zip(*vfam_protein_data)
                
                # create a df for vfam
                vfam_df = pd.DataFrame({
                    "Group": vfam_groups,
                    "Protein Name": vfam_protein_names,
                    "UniProt ID": vfam_accessions,
                    "Source": "VFAM" 
                })
                
                # remove duplicate rows if necessary
                vfam_df = vfam_df.drop_duplicates()
    except Exception as e:
        print(f"error processing vfam data: {e}")

# process vog data
if len(vog_names) > 0:
    try:
        # read the vog annotations df
        vog_annotations = pd.read_csv('/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/phmms/vog.annotations.tsv', sep='\t')
        
        # filter the vog_annotations 
        filtered_vog_annotations = vog_annotations[vog_annotations['#GroupName'].isin(vog_names)].copy()
        
        if not filtered_vog_annotations.empty:
            # extract information for vog
            vog_protein_data = filtered_vog_annotations.apply(
                lambda row: (
                    row['#GroupName'],  # vog group
                    row['ConsensusFunctionalDescription'].split('|')[1] if '|' in row['ConsensusFunctionalDescription'] and len(row['ConsensusFunctionalDescription'].split('|')) > 1 else 'not found',  # accession id
                    row['ConsensusFunctionalDescription'].split('|')[2].split(' ')[0] if '|' in row['ConsensusFunctionalDescription'] and len(row['ConsensusFunctionalDescription'].split('|')) > 2 else 'not found'  # protein name
                ),
                axis=1
            ).values.tolist()
            
            if vog_protein_data:
                # convert list of tuples into three separate lists
                vog_groups, vog_accessions, vog_protein_names = zip(*vog_protein_data)
                
                # create a df for vog
                vog_df = pd.DataFrame({
                    "Group": vog_groups,
                    "Protein Name": vog_protein_names,
                    "UniProt ID": vog_accessions,
                    "Source": "VOG"
                })
                
                # remove duplicate rows if necessary
                vog_df = vog_df.drop_duplicates()
    except Exception as e:
        print(f"error processing vog data: {e}")

# concatenate the two dataframes to create unified_df
unified_df = pd.concat([vfam_df, vog_df], ignore_index=True)

print("\nunified dataframe summary:")
print(f"total entries: {len(unified_df)}")
print(f"vfam entries: {len(vfam_df)}")
print(f"vog entries: {len(vog_df)}")
print(f"unique protein names: {unified_df['Protein Name'].nunique()}")
print(f"unique uniprot ids: {unified_df['UniProt ID'].nunique()}")

In [12]:
import requests
import time
import pandas as pd

# 1. function to fetch go ids
def get_go_ids_only(uniprot_id):
    """
    fetches go ids from quickgo for a given uniprot id
    
    parameters:
    uniprot_id (str): the uniprot accession number
    
    returns:
    str: a comma-separated list of go ids or an error message
    """
    url = f"https://www.ebi.ac.uk/QuickGO/services/annotation/search?geneProductId={uniprot_id}"
    headers = {"accept": "application/json"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        # extract the go ids
        go_ids = set()
        if 'results' in data:
            for entry in data['results']:
                go_id = entry.get('goId', '')
                if go_id and go_id.startswith('GO:'):
                    go_ids.add(go_id)
        
        # return a simple comma-separated list of go ids
        return ", ".join(sorted(go_ids)) if go_ids else "no go terms found"
    except requests.exceptions.RequestException:
        return "error fetching go terms"

# 2. function to get go aspect
def get_go_aspect(go_id):
    """
    retrieves the aspect of a go term (mf, cc, bp)
    
    parameters:
    go_id (str): the go term id (e.g., 'go:0044423')
    
    returns:
    str: the aspect code(s) of the go term or an error message
    """
    print(f"fetching aspect for {go_id}...")
    url = f"https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/{go_id}"
    headers = {"accept": "application/json"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        # extract all aspects (some terms can have multiple)
        aspects = data.get('results', [{}])[0].get('aspect', 'unknown')
        
        # map aspects to mf, cc, bp
        aspect_mapping = {
            "molecular_function": "mf",
            "cellular_component": "cc",
            "biological_process": "bp"
        }
        
        # convert to list (if multiple aspects exist)
        if isinstance(aspects, list):
            mapped_aspects = [aspect_mapping.get(aspect.lower(), "unknown") for aspect in aspects]
        else:
            mapped_aspects = [aspect_mapping.get(aspects.lower(), "unknown")]
        
        result = ", ".join(sorted(set(mapped_aspects))) if mapped_aspects else "unknown"
        print(f"found aspect for {go_id}: {result}")
        return result
    except requests.exceptions.RequestException:
        print(f"error fetching aspect for {go_id}")
        return "not found"

# 3. fetch go terms for each uniprot id
print("fetching go terms for each uniprot id...")
go_id_results = {}

# process each uniprot id to fetch go terms
for idx, row in unified_df.iterrows():
    uniprot_id = row['UniProt ID']
    
    # skip if the uniprot id is not found or invalid
    if uniprot_id == 'not found':
        continue
    
    print(f"fetching go terms for uniprot id: {uniprot_id}")
    
    # add small delay between api calls
    time.sleep(0.3)
    
    # fetch go terms
    go_terms = get_go_ids_only(uniprot_id)
    go_id_results[uniprot_id] = go_terms
    
    print(f"found go terms for {uniprot_id}: {go_terms}")

print(f"go term fetching complete! processed {len(go_id_results)} uniprot ids")

# 4. process go terms to add aspect information
print("enhancing go terms with aspect information...")
print(f"total uniprot ids to process: {len(go_id_results)}")

# create a dictionary to store enhanced go term information
enhanced_go_info = {}
processed_count = 0
total_terms_processed = 0

# process each uniprot id that has go terms
for uniprot_id, go_terms_str in go_id_results.items():
    processed_count += 1
    print(f"processing uniprot id {processed_count}/{len(go_id_results)}: {uniprot_id}")
    
    if go_terms_str in ["no go terms found", "error fetching go terms"]:
        print("no go terms found for this id")
        enhanced_go_info[uniprot_id] = go_terms_str
        continue
        
    # split the comma-separated go terms
    go_terms = [term.strip() for term in go_terms_str.split(',')]
    print(f"found {len(go_terms)} go terms for this id")
    
    # process each go term
    enhanced_terms = []
    for i, term in enumerate(go_terms):
        total_terms_processed += 1
        print(f"processing term {i+1}/{len(go_terms)}: {term}")
        
        # add small delay between api calls
        time.sleep(0.3)
        
        # get aspect
        aspect = get_go_aspect(term)
        
        # format the enhanced term
        enhanced_term = f"{term} ({aspect})"
        enhanced_terms.append(enhanced_term)
        print(f"enhanced term: {enhanced_term}")
    
    # store the enhanced information
    enhanced_go_info[uniprot_id] = "; ".join(enhanced_terms)
    print(f"completed processing uniprot id: {uniprot_id}")

print("go term enhancement complete!")
print(f"total uniprot ids processed: {processed_count}")
print(f"total go terms processed: {total_terms_processed}")

# 5. update the dataframe with enhanced go term information
print("adding go terms with aspects to dataframe...")
unified_df['GO Terms with Aspect'] = unified_df['UniProt ID'].apply(
    lambda acc: enhanced_go_info.get(acc, "no go terms found") if acc != 'not found' else "not applicable"
)

# 6. save the enhanced dataframe
enhanced_output_file = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/groups/goterms.csv"
unified_df.to_csv(enhanced_output_file, index=False)
print(f"enhanced results saved to {enhanced_output_file}")
print("processing complete!")

Fetching GO terms for each UniProt ID...
Fetching GO terms for UniProt ID: Q83050
Found GO terms for Q83050: GO:0019028, GO:0044423
Fetching GO terms for UniProt ID: A1XIP9
Found GO terms for A1XIP9: GO:0001172, GO:0003723, GO:0003724, GO:0003968, GO:0004197, GO:0004386, GO:0005524, GO:0006260, GO:0006351, GO:0006508, GO:0008234, GO:0016020, GO:0033644, GO:0034062, GO:0039694
Fetching GO terms for UniProt ID: Q9QBU3
Found GO terms for Q9QBU3: GO:0005198, GO:0019028, GO:0039617
Fetching GO terms for UniProt ID: Q6EWG8
Found GO terms for Q6EWG8: GO:0019028, GO:0044156, GO:0044219, GO:0046740
Fetching GO terms for UniProt ID: Q9YX37
Found GO terms for Q9YX37: no go terms found
Fetching GO terms for UniProt ID: Q86134
Found GO terms for Q86134: GO:0003723, GO:0019013, GO:0019029, GO:0030430, GO:1990904
Fetching GO terms for UniProt ID: Q6Q304
Found GO terms for Q6Q304: GO:0016020, GO:0033644, GO:0044177, GO:0044178, GO:0044423, GO:0055036
Fetching GO terms for UniProt ID: P10357
Found GO t

# Example usages

In [ ]:
# Example 1: Run taxonomic evaluation
"""
# Define input files
feature_files = [
    "/path/to/features1.csv",
    "/path/to/features2.csv"
]
y_path = "/path/to/labels.csv"
output_folder = "/path/to/output_directory"
rank_paths = {
    "Family": "/path/to/family_ranks.csv",
    "Order": "/path/to/order_ranks.csv",
    "Genus": "/path/to/genus_ranks.csv"
}

# Run the taxonomic evaluation
results, combined_df = run_taxonomic_evaluation(
    feature_files=feature_files,
    y_path=y_path,
    output_folder=output_folder,
    rank_paths=rank_paths
)
"""
# Example 2: Run feature selection analysis
"""
# Define input files
feature_files = [
    "/path/to/features1.csv",
    "/path/to/features2.csv"
]
y_path = "/path/to/labels.csv"
output_folder = "/path/to/feature_selection_results"

# Run the feature selection with multiple alpha values
run_feature_selection_analysis(
    feature_files=feature_files,
    y_path=y_path,
    output_folder=output_folder,
    alphas=[0, 0.001, 0.005, 0.01, 0.05, 0.1],  # custom alpha values for feature selection
    n_splits=10  # number of cross-validation folds
)
"""